In [2]:
import pandas as pd
from pathlib import Path

# caminho do arquivo
file_path = Path("../data/db_case_sales_ops.xlsx")

# carregar todas as abas
lead = pd.read_excel(file_path, sheet_name="lead")
visitas = pd.read_excel(file_path, sheet_name="visitas")
contratos = pd.read_excel(file_path, sheet_name="contratos")
geral = pd.read_excel(file_path, sheet_name="geral")

# padronizar datas em todas as tabelas
def padroniza_datas(df):
    for col in df.columns:
        col_str = str(col).lower()  
        if "data" in col_str:
            df[col] = pd.to_datetime(df[col], errors="coerce")
    return df


lead = padroniza_datas(lead)
visitas = padroniza_datas(visitas)
contratos = padroniza_datas(contratos)
geral = padroniza_datas(geral)


In [ ]:
# LIMPEZAA PLANILHA GERAL

# 1. Remover linhas totalmente vazias
geral = geral.dropna(how="all")

# 2. Remover colunas totalmente vazias 
geral = geral.dropna(axis=1, how="all")

# 3. Derreter (melt) para formato longo
geral_long = geral.melt(
    id_vars="metrica",
    var_name="mes",
    value_name="valor"
)

# 4. Remover linhas sem mês ou sem métrica
geral_long = geral_long.dropna(subset=["mes", "metrica"])

# 5. Converter mes para datetime
geral_long["mes"] = pd.to_datetime(geral_long["mes"], format="%Y-%m")

# 6. Padronizar nome das métricas
geral_long["metrica"] = (
    geral_long["metrica"]
    .str.lower()
    .str.strip()
    .replace({
        "atingimento": "atingimento_vendas",
        "vendas": "vendas",
        "visitas": "visitas",
        "meta": "meta_vendas"
    })
)

# 7. Converter valores para numérico
geral_long["valor"] = pd.to_numeric(geral_long["valor"], errors="coerce")

geral_long.head()


,metrica,mes,valor
0,metas,2024-04-01,NaN
1,vendas,2024-04-01,625.0
2,visitas realizadas,2024-04-01,875.0
3,realizado,2024-04-01,NaN
4,vendas,2024-04-01,548.0


In [ ]:
import numpy as np


# Renomear colunas
lead = lead.rename(columns={
    "id": "lead_id",
    "data_criacao": "data_criacao",
    "data_perda": "data_perda",
    "data_venda": "data_venda",
    "utm_source": "utm_source",
    "sdr": "sdr_responsavel",
    "closer": "closer_responsavel",
    "profissao": "profissao",
    "indicado": "indicado",
    "tipo": "tipo_lead",
    "motivoPerda": "motivo_perda"
})


# 1. Conversão e Padronização Geral

# Converter datas
date_cols = ["data_criacao", "data_perda", "data_venda"]
for col in date_cols:
    lead[col] = pd.to_datetime(lead[col], errors="coerce")

# Padronizar texto e nulos em outras colunas (mais limpo com fillna)
str_cols = ["utm_source", "sdr_responsavel", "closer_responsavel",
            "profissao", "tipo_lead", "motivo_perda"]

for col in str_cols:
    lead[col] = (
        lead[col]
        .fillna('none')
        .astype(str).str.lower().str.strip()
        .replace(['nan', 'none', ''], np.nan)
    )

# 2. 🎯 Padronização da coluna 'indicado' 
lead["indicado"] = (
    lead["indicado"]
    .astype(str).str.lower().str.strip()
    .replace('nan', 'não informado') 
)


# 3. Criar coluna status geral do lead (Usando np.where para concisão)
lead["status_final"] = np.where(
    lead["data_venda"].notna(), 
    "VENDIDO",
    np.where(
        lead["data_perda"].notna(), 
        "PERDIDO", 
        "EM_ABERTO"
    )
)


# --- Criação do DataFrame Final e Métricas ---

# O DF 'leads' é o resultado final da limpeza base, removendo duplicatas
leads = lead.drop_duplicates(subset="lead_id", keep="first").copy()


# Flags e Métricas
# Flag de indicado (AGORA É BASEADO EM STRING: 'indicado' -> 1, outros -> 0)
# Usa np.where para criar a flag booleana (1/0) a partir da string padronizada
leads["flag_indicado"] = np.where(
    leads["indicado"] == 'indicado',
    1,
    0
)

# Flag de venda concluída
leads["flag_venda"] = leads["data_venda"].notna().astype(int)

# Métricas de ciclo (Dias)
leads["dias_ate_venda"] = (leads["data_venda"] - leads["data_criacao"]).dt.days
leads["dias_ate_perda"] = (leads["data_perda"] - leads["data_criacao"]).dt.days



Limpeza da tabela contratos

In [17]:
visitas.columns.tolist()

['id', 'data_visita', 'tipoAgendamento', 'ferramentaAgendamento', 'unidade']

In [10]:
# renomear colunas
contratos = contratos.rename(columns={
    "id": "contrato_id",
    "dataVenda": "data_venda",
    "modalidadeContrato": "modalidade_contrato"
})

# converter datas
contratos["data_venda"] = pd.to_datetime(contratos["data_venda"], errors="coerce")

# remover duplicados
contratos = contratos.drop_duplicates(subset="contrato_id", keep="first")

# remover contratos sem data
contratos = contratos[contratos["data_venda"].notna()]

# padronizar textos
contratos["modalidade_contrato"] = contratos["modalidade_contrato"].str.lower().str.strip()
contratos["plano"] = contratos["plano"].str.lower().str.strip()


In [ ]:
# LIMPEZA PLANILHA DE VISITAS 

# 1. Renomear colunas para formato padronizado
visitas = visitas.rename(columns={
    "id": "id_visita",
    "data_visita": "data_visita",
    "tipoAgendamento": "tipo_agendamento",
    "ferramentaAgendamento": "ferramenta_agendamento",
    "unidade": "unidade"
})


# 2. Converter datas

visitas["data_visita"] = pd.to_datetime(visitas["data_visita"], errors="coerce")

# Remover linhas onde a data é nula imediatamente (não servem para análise temporal)
visitas = visitas.dropna(subset=["data_visita"])


# 3. Engenharia de Features

# Cria uma coluna de Mês/Ano (ex: "2024-08") para cruzar graficamente com a safra de leads
visitas["mes_referencia"] = visitas["data_visita"].dt.to_period("M").astype(str)

# 4. Padronizar colunas de texto

str_cols = ["tipo_agendamento", "ferramenta_agendamento", "unidade"]

for col in str_cols:
    visitas[col] = (
        visitas[col]
            .astype(str)
            .str.lower()
            .str.strip()
            .replace(["nan", "none"], None) # Garante limpeza de nulos string
    )


# 5. Remover duplicados

# Garante que uma mesma visita não foi registrada duas vezes
visitas = visitas.drop_duplicates(subset=["id_visita"], keep="first")

# 6. Seleção e Ordenação Final

visitas = visitas[[
    "id_visita",
    "data_visita",
    "mes_referencia",
    "unidade",
    "tipo_agendamento",
    "ferramenta_agendamento"
]].sort_values(by="data_visita", ascending=False)

# Preview final
print(f"Total de Visitas Limpas: {len(visitas)}")
visitas.head()

Total de Visitas Limpas: 9115


,id_visita,data_visita,mes_referencia,unidade,tipo_agendamento,ferramenta_agendamento
9561,125127,2025-04-30,2025-04,visita virtual,visita agendada,admin
9535,133195,2025-04-30,2025-04,livance - tatuapé,visita agendada,animus
9533,127371,2025-04-30,2025-04,livance - santos,visita agendada,retool
9532,133005,2025-04-30,2025-04,livance - santos,visita agendada,retool
9531,133042,2025-04-30,2025-04,livance - santos,visita agendada,retool


In [ ]:
Path("../processed").mkdir(exist_ok=True)

contratos.to_csv("../processed/contratos_limpo.csv", index=False, encoding="utf-8")
geral.to_csv("../processed/geral_limpo.csv", index=False, encoding="utf-8")
lead.to_csv("../processed/leads_limpo.csv", index=False, encoding="utf-8")



Arquivos salvos com sucesso na pasta 'processed'!


lead.to_csv("../processed/leads_limpo.csv", index=False, encoding="utf-8")


In [31]:
visitas.to_csv("../processed/visitas_limpo.csv", index=False, encoding="utf-8")
